# Naive Bayes Learner

In [68]:
# Load libraries
import numpy as np
import pandas as pd
from collections import defaultdict
from math import log, exp

## Implementation

In [69]:
def preprocess(csv_path):
    """Reads and processes a csv data file. Returns a tuple of:
    (<2D list of instances>, <list of class labels>, <number of unique labels>).
    """
    
    df = pd.read_csv(csv_path, header=None)

    # Add a list of each instance for each attribute (the first N-1 columns in the DataFrame)
    instance_list = []
    if ((len(df.columns) > 1)):
        for attribute_index in range(0, (len(df.columns) - 2)):
            instance_list.append(df[attribute_index].tolist())
    
    # Make sure attribute instances are in String format
    for index in range (0, len(instance_list)):
        instance_list[index] = [str(i) for i in instance_list[index]]
        
    class_list = []
    if ((len(df.columns) > 0)):
        class_list = df[(len(df.columns) - 1)].tolist()
    class_list = [str(i) for i in class_list]
    
    n_classes = len(set(class_list))
    return instance_list, class_list, n_classes

In [70]:
def train_supervised(data):
    """Trains a supervised Naive Bayes learner based on a processed csv file.
    
    Takes in data as a tuple of (<list of instances>, <list of class labels>).
    Uses Laplace Smoothing to make all posterior probabilities non-zero.
    It returns a supervised Naive Bayes model as a tuple of of prior and posterior counts.
    """

    def calculate_priors(class_list):
        """calculate_priors takes in a list of class labels in String format.
        It returns a dictonary of class labels and their prior probabilities
        """ 
        
        prior_dd = defaultdict(int)
        
        # Count all class labels and add to dictionary
        for class_label in class_list:
            prior_dd[class_label] += 1
            
        return dict(prior_dd)
  

    def calculate_posteriors(priors_dict, instance_list, class_list):
        """calculate_posteriors takes in a dictionary or prior probabilities for class labels,
        a 2D list of instance values for each attribute and a list of class labels.
        
        It returns a list of 2D dictionaries containing the posterior probabilities
        for each attrbute possibility in each attribute column, given a class label
        """
        
        n_instances = len(class_list)

        # Initialize a list of dictionaries, one for each attribute
        posteriors_dictlist = [dict() for x in range(len(instance_list))]

        # Initialize a default dict for each class label, for each attribute dictionary
        for attribute_dd in posteriors_dictlist:
            for class_label in priors_dict.keys():
                # Start at 1 for Laplace smoothing
                attribute_dd[class_label] = defaultdict(lambda:1)


        # Count the number of instances for each conditional probability P(Attribute=attr_instance | Class)
        for col in range(len(instance_list)):
            for row in range(len(instance_list[col])):
                
                #Format is <Attribute> <Class> <Attribute Instance>
                posteriors_dictlist[col][class_list[row]][instance_list[col][row]] += 1
            
            # Keep track of all attribute possibilites
            attr_set = set()
            for label in posteriors_dictlist[col].keys():
                for attr in posteriors_dictlist[col][label].keys():
                    attr_set.add(attr)
            
            # Add attributes with counts of 1 (Laplace Smoothing) when no occurances for a given class
            for label in posteriors_dictlist[col].keys():
                for attr in attr_set:
                    if attr not in posteriors_dictlist[col][label].keys():
                        # Start at 1 for Laplace smoothing
                        posteriors_dictlist[col][label][attr] = 1
                        
        return posteriors_dictlist

    # A dictionary containing the prior probabilities of class_labels,
    priors_dict = calculate_priors(data[1])
    
    # Initialize a list of dictionaries, one for each attribute of the dataset
    # The index of the dictlist corresponds to the attribute index of instance_list, or data[0]
    posteriors_dictlist = calculate_posteriors(priors_dict, data[0], data[1])
    
    return priors_dict, posteriors_dictlist

In [71]:
def predict_supervised(test_set, NB_model):
    """Predicts the class for a set of instances, based on a trained supervised Naive Bayes model.
    
    Uses Laplace Smoothing to make all posterior probabilities non-zero.
    Returns a list of class predictions.
    """
    
    predictions = []
    
    n_test_instances = len(test_set[0])
    priors = NB_model[0]
    posteriors = NB_model[1]
    
    # Make a prediction for every instance in the test set
    for test_row in range(n_test_instances):
        label_predict_probs = []
        
        # Calculate prediction probability for each class label
        for label in priors.keys():
            label_count = priors[label]
            
            # Prior log probability log(P(label))
            label_prob = log(label_count / n_test_instances)
            
            # Sum the prediction probability and log(posterior probabilities) to avoid underlow
            # Dividing by the number of labels + number of attribute values (Laplace Smoothing)
            for test_col in range(len(test_set)):
                attr = test_set[test_col][test_row]
                
                posterior_prob = posteriors[test_col][label][attr] / \
                        (label_count + len(posteriors[test_col][label]))
                
                label_prob += log(posterior_prob)
            
            # Turn log probabilitiy back in probability
            label_prob = exp(label_prob)
            label_predict_probs.append((label_prob, label))

        # Sort the predictions from high-low and predict the label with the highest probability
        label_predict_probs.sort(reverse=True)
        predictions.append(label_predict_probs[0][1])
    
    return predictions

In [72]:
def evaluate_supervised(predicted_classes, actual_classes):
    """Evaluates the number of correct predictions made by a supervised classifier.
    Returns an accuracy score between [0,1].
    """
    
    n_correct = 0
    for test in range(len(predicted_classes)):
        if predicted_classes[test] == actual_classes[test]:
            n_correct += 1
    return n_correct / len(predicted_classes)

## Results

In [73]:
def test_and_print_supervised(dataset_csv_path):
    """Trains and evaluates a supervised Naive Bayes learner and prints an accuracy score"""
    
    data = preprocess(dataset_csv_path)
    model = train_supervised(data)
    predicted_classes = predict_supervised(data[0], model)   
    acc = evaluate_supervised(predicted_classes, data[1])
    print('Acc: '+ '{0:.2f}'.format(acc * 100) + '% for ' + dataset_csv_path.split('/')[-1] \
          +  ' with ' + str(len(predicted_classes)) + ' instances')

In [74]:
# Print supervised results with laplace smoothing
print('Supervised Naive Bayes (Laplace Smoothing)')
print('------------------------------------------')
test_and_print_supervised('data/breast-cancer.csv')
test_and_print_supervised('data/car.csv')
test_and_print_supervised('data/hypothyroid.csv')
test_and_print_supervised('data/mushroom.csv')

Supervised Naive Bayes (Laplace Smoothing)
------------------------------------------
Acc: 75.52% for breast-cancer.csv with 286 instances
Acc: 69.85% for car.csv with 1728 instances
Acc: 95.19% for hypothyroid.csv with 3163 instances
Acc: 95.53% for mushroom.csv with 8124 instances
